In [2]:
import earthdata
import xarray
import pyproj
import numpy as np
from osgeo import osr
import rasterio
from rasterio.crs import CRS
from rasterio.transform import Affine
from rasterio.warp import calculate_default_transform

In [3]:
# We import the classes from earthdata
from earthdata import Auth, DataCollections, DataGranules, Store
auth = Auth().login(strategy="interactive")
#auth = Auth().login(strategy="netrc")

Enter your Earthdata Login username: aimeeb
Enter your Earthdata password: ········
You're now authenticated with NASA Earthdata Login


In [95]:
Query = DataGranules().short_name('IS2SITMOGR4').temporal("2021-01-01", "2021-01-31")
granules = Query.get(1)
print(granules[0])

{'meta': {'concept-type': 'granule', 'concept-id': 'G2113544897-NSIDC_ECS', 'revision-id': 1, 'native-id': 'SC:IS2SITMOGR4.001:221651682', 'provider-id': 'NSIDC_ECS', 'format': 'application/echo10+xml', 'revision-date': '2021-09-02T19:46:35.988Z'}, 'umm': {'TemporalExtent': {'RangeDateTime': {'BeginningDateTime': '2021-01-01T00:00:00.000Z', 'EndingDateTime': '2021-01-31T23:59:59.990Z'}}, 'GranuleUR': 'SC:IS2SITMOGR4.001:221651682', 'AdditionalAttributes': [{'Name': 'SIPSMetGenVersion', 'Values': ['2.5']}], 'SpatialExtent': {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -180.0, 'EastBoundingCoordinate': 180.0, 'NorthBoundingCoordinate': 89.0, 'SouthBoundingCoordinate': 30.0}]}}}, 'ProviderDates': [{'Date': '2021-09-02T13:44:30.278Z', 'Type': 'Insert'}, {'Date': '2021-09-02T13:45:02.452Z', 'Type': 'Update'}], 'CollectionReference': {'EntryTitle': 'ICESat-2 L4 Monthly Gridded Sea Ice Thickness V001'}, 'RelatedUrls': [{'URL': 'https://n5eil01u.e

In [96]:
granules[0].cloud_hosted

False

In [97]:
store = Store(auth)

In [98]:
data_dir = './data'
files = store.get(granules[0:], data_dir)

 Getting 1 granules, approx download size: 0.01 GB


SUBMITTING | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING | :   0%|          | 0/1 [00:00<?, ?it/s]

In [99]:
import glob
local_files = glob.glob(f'{data_dir}/*.nc')
local_files
icesat2_file = local_files[0]

In [100]:
icesat2_file

'./data/IS2SITMOGR4_01_202101_004_001.nc'

In [104]:
from pathlib import Path

stem = Path(icesat2_file).stem
stem

'IS2SITMOGR4_01_202101_004_001'

In [105]:
proj4326 = pyproj.Transformer.from_crs("epsg:3413", "epsg:4326", always_xy=True)
proj3857 = pyproj.Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)

xds = xarray.open_dataset(icesat2_file)

ice_thickness = xds.ice_thickness

In [106]:
bounds = (-3850000.0, -5350000.0, 3750000.0, 5850000.0)

transform, width, height = calculate_default_transform(
    CRS.from_epsg(3411),
    CRS.from_epsg(3411),
    ice_thickness.shape[0],
    ice_thickness.shape[1],
    *bounds
)

print(transform, width, height)

profile = {
    'driver': 'GTiff',
    'dtype': 'float32',
    'nodata': 0,
    'width': width,
    'height': height,
    'count': 1,
    'crs': CRS.from_epsg(3411),
    'transform': transform,
    'tiled': False,
    'interleave': 'band'
}
val = np.where(np.isnan(ice_thickness))
ice_thickness = np.array(ice_thickness)
ice_thickness[val] = 0

tif_filename = f'{data_dir}/{stem}'
with rasterio.open(f'{tif_filename}.tif', 'w', **profile) as tif:
    tif.write(ice_thickness, 1)

| 25000.00, 0.00,-3850000.00|
| 0.00,-25000.00, 5850000.00|
| 0.00, 0.00, 1.00| 304 448


In [107]:
# sentinel hub needs the full datetime
yearmonth = stem.split('_')[2]
beginning_datetime = granules[0]['umm']['TemporalExtent']['RangeDateTime']['BeginningDateTime']
beginning_datetime
new_filename = f'{data_dir}/{stem.replace(yearmonth, beginning_datetime)}'
new_filename

'./data/IS2SITMOGR4_01_2021-01-01T00:00:00.000Z_004_001'

In [108]:
tif_filename = f'{data_dir}/{stem}'

# Project to 4326
!gdalwarp -overwrite -t_srs EPSG:4326 -te -180 31.103 179.814 89.837 {tif_filename}.tif {tif_filename}_4326.tif

# Clip to valid bounds for 3857 
# -projwin <ulx> <uly> <lrx> <lry>
!gdal_translate -projwin -180 85.0 179.814 31.103 \
  {tif_filename}_4326.tif {tif_filename}_clipped_for_3857.tif

!gdalwarp -overwrite \
  -s_srs epsg:4326 -t_srs epsg:3857 \
  {tif_filename}_clipped_for_3857.tif {tif_filename}_3857.tif
        
!gdaladdo -r average --config GDAL_TIFF_OVR_BLOCKSIZE 1024 {tif_filename}_3857.tif 2 4 8 16 32
   
!gdal_translate -co TILED=YES -co COPY_SRC_OVERVIEWS=YES \
  --config GDAL_TIFF_OVR_BLOCKSIZE 1024 -co BLOCKXSIZE=1024 -co BLOCKYSIZE=1024 -co COMPRESS=DEFLATE \
  {tif_filename}_3857.tif  {new_filename}_COG.tif

Creating output file that is 1704P x 278L.
Processing ./data/IS2SITMOGR4_01_202101_004_001.tif [1/1] : 0Using internal nodata values (e.g. 0) for image ./data/IS2SITMOGR4_01_202101_004_001.tif.
Copying nodata values from source ./data/IS2SITMOGR4_01_202101_004_001.tif to destination ./data/IS2SITMOGR4_01_202101_004_001_4326.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 1704, 278
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1592P x 658L.
Processing ./data/IS2SITMOGR4_01_202101_004_001_clipped_for_3857.tif [1/1] : 0Using internal nodata values (e.g. 0) for image ./data/IS2SITMOGR4_01_202101_004_001_clipped_for_3857.tif.
Copying nodata values from source ./data/IS2SITMOGR4_01_202101_004_001_clipped_for_3857.tif to destination ./data/IS2SITMOGR4_01_202101_004_001_3857.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size 

In [109]:
final_file = f'{new_filename}_COG.tif'
!rio cogeo validate {final_file}

/Users/aimeebarciauskas/github/nasa-impact/cloud-optimized-data-pipelines/dataset-workflows/IS2SITMOGR4/data/IS2SITMOGR4_01_2021-01-01T00:00:00.000Z_004_001_COG.tif is a valid cloud optimized GeoTIFF
